In [1]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath="./csv_files/"

In [2]:
# 生データを引数として受け取り、そのデータの問題サイズの値に合わせた追加の初期値を追加し、引数として渡されたDFにデータが付与されたDFを返す関数
# 引数：生データDF
# 返り値：生データDFにベンチマークごとの初期値が付与されたDF


def addInitDataToRawDF(rawDF):
    return rawDF


def test_addInitDataToRawDF():
    # rawDF の作成
    functionName = [
        "functionAtCG",
        "functionAtEP",
        "functionNameAtFP",
        "functionNameAtIS",
        "functionNameAtLU",
        "functionNameAtMG",
    ]
    functionCallNum = [1, 2, 3, 4, 5, 6]
    benchmarkName = ["cg", "ep", "fp", "is", "lu", "mg"]
    benchmarkClass = ["A", "B", "C", "D", "E", "F"]
    process = [2, 4, 8, 16, 32, 64]
    rawDict = {
        "functionName": functionName,
        "functionCallNum": functionCallNum,
        "benchmarkName": benchmarkName,
        "benchmarkName": benchmarkName,
        "process": process,
    }

    rawDF = pd.DataFrame
    # rawDF に手動で関数のやっていることを実施した DF を作成
    # 比較して正しいことを確認
    pass


sampleRawDF = returnCollectedExistingData(
    benchmarkNames=["cg"],
    classes=["A", "B", "C", "D", "E", "F"],
    processes=[2, 4, 8, 16, 32, 64, 128, 256],
    csvDirPath=csvDirPath,
)

test_addInitDataToRawDF()

sampleRawDF

,functionName,functionCallNum,benchmarkName,benchmarkClass,process
0,.TAU_application,1.0,cg,A,2
1,CG,1.0,cg,A,2
2,MPI_Finalize(),1.0,cg,A,2
3,INITIALIZE_MPI,1.0,cg,A,2
4,MPI_Init(),1.0,cg,A,2
...,...,...,...,...,...
17,MPI_Reduce(),1.0,cg,F,256
18,SETUP_SUBMATRIX_INFO,1.0,cg,F,256
19,MPI_Comm_size(),1.0,cg,F,256
20,MPI_Comm_rank(),1.0,cg,F,256
